<a href="https://colab.research.google.com/github/p4arth/NLP/blob/main/ImplementingFakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/LstmFake/train.csv')
test = pd.read_csv('/content/drive/MyDrive/LstmFake/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df=df.dropna()

In [ ]:
x=df.drop('label',axis=1)
y = df['label']

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
voc_size = 5000

In [ ]:
messages=x.copy()
messages.reset_index(inplace=True)

In [ ]:
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = " ".join(review)
  corpus.append(review)

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot = [one_hot(words,voc_size) for words in corpus]
onehot

[[668, 2598, 2115, 3870, 4660, 1345, 1508, 4344, 932, 505],
 [2644, 1075, 1785, 1034, 134, 4365, 1339],
 [3659, 3099, 801, 4290],
 [1845, 94, 113, 1928, 880, 3750],
 [1389, 134, 3447, 577, 4103, 4647, 134, 4385, 2034, 2276],
 [2152,
  4332,
  3114,
  2343,
  3198,
  2605,
  1605,
  1538,
  1046,
  1691,
  115,
  4614,
  531,
  4796,
  1339],
 [3635, 4726, 2878, 3382, 3280, 2924, 4883, 330, 2846, 1848, 1175],
 [838, 3908, 1759, 3844, 1260, 3619, 2605, 4977, 2846, 1848, 1175],
 [682, 1747, 1769, 3870, 4883, 3355, 2276, 3171, 2605, 1966],
 [3580, 213, 1790, 3044, 1468, 568, 593, 352],
 [3953, 3416, 1073, 1815, 3750, 1734, 1089, 4166, 4904, 415, 4621],
 [1928, 500, 4660, 3355, 2605, 1260],
 [4309, 1986, 315, 935, 1570, 2671, 1579, 2380, 4056],
 [2566, 2484, 1063, 526, 4441, 2702, 3046, 2846, 1848, 1175],
 [3127, 2661, 2580, 1638, 4899, 2846, 1848, 1175],
 [4806, 3779, 2505, 2279, 1690, 1479, 1025, 2398, 1742, 3062],
 [2799, 4691, 1075],
 [2209, 3140, 1800, 4926, 2605, 2590, 4971, 1339],
 [

In [ ]:
sent_length = 20
embedded = pad_sequences(onehot, padding='pre', maxlen=sent_length)

In [ ]:
from tensorflow.keras.layers import Dropout
embeding_features = 40
model = Sequential()
model.add(Embedding(voc_size,embeding_features, input_length=sent_length))

model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
x_final = np.array(embedded)
y_final = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [ ]:
model.fit(x_train,y_train,validation_data = (x_test,y_test), epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 14ms/step - loss: 0.3348 - accuracy: 0.8432 - val_loss: 0.1910 - val_accuracy: 0.9196
Epoch 2/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1347 - accuracy: 0.9465 - val_loss: 0.2041 - val_accuracy: 0.9241
Epoch 3/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0941 - accuracy: 0.9661 - val_loss: 0.2227 - val_accuracy: 0.9183
Epoch 4/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0740 - accuracy: 0.9727 - val_loss: 0.2239 - val_accuracy: 0.9155
Epoch 5/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0564 - accuracy: 0.9803 - val_loss: 0.2610 - val_accuracy: 0.9120
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0435 - accuracy: 0.9867 - val_loss: 0.3769 - val_accuracy: 0.9127
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0305 - accuracy: 0.9903 - val_loss: 0.3474 - val_accuracy: 0.9145

In [ ]:
y_pred=model.predict(x_test)
classes = np.argmax(y_pred,axis=1)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,classes)

array([[3419,    0],
       [2616,    0]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,classes)

0.5665285832642917